In [15]:
access_token = 'ntc74JBCAkBLNUglVKHl0vyawpu9dzOk6X_krDgpdrRUVTsVc5V5TWLwHd79DOlz'

In [9]:
import requests

base_url = "http://api.genius.com"
#Key line below here when, this is how to authorize your request when
#using the API
headers = {'Authorization': 'Bearer ' + access_token}
search_url = base_url + "/search"
song_title = "The Serpent is Kind (Compared to Man)"
params = {'q': song_title}
response = requests.get(search_url, params=params, headers=headers)

In [34]:
import lyricsgenius as genius
api = genius.Genius(access_token)
artist = api.search_artist('Hiss Golden Messenger', max_songs=10, get_full_song_info=False)

Searching for songs by Hiss Golden Messenger...

Song 1: "Ace of Cups Hung Low Band"
Song 2: "As the Crow Flies"
Song 3: "Bad Debt"
Song 4: "Balthazar's Song"
Song 5: "Biloxi"
Song 6: "Black Dog Wind (Rose of Roses)"
Song 7: "Blue Country Mystic"
Song 8: "Brother, Do You Know the Road?"
Song 9: "Brother, Do You Know the Road? by Hiss Golden Messenger"
Song 10: "Brown-Eyed Women"

Reached user-specified song limit (10).
Found 10 songs.
Done.


In [17]:
print(artist)

Hiss Golden Messenger, 3 songs


In [18]:
artist.name

'Hiss Golden Messenger'

In [27]:
typeartist.songs[2]

lyricsgenius.song.Song

In [30]:
song = api.search_song('The Serpent is Kind (Compared to Man)')

Searching for "The Serpent is Kind (Compared to Man)"...
Done.


In [33]:
print(song.lyrics)

I turn my face to the waterline
I turn my face far away
This, I realize, is not the way
Where the dark lies, push it away

[Chorus]
Hard by design
Hard times
Are just a drop of rain
So let the rain come on
Behind, behind runs the sun

I put my hands on the red dirt land
I work the land where my father tread
He said 'Don't be afraid when the snake is at hand;
The serpent is kind compared to man'

[Chorus]
Hard by design
Hard times
Are just a drop of rain
Let the rain come on
Behind, behind runs the sun

The littlest light
Is the break of day
The littlest lamp
Finds a way
All of our friends
Across the land
Alive in the light
And I called her then

[Chorus x2]
Hard by design
Hard times
Are just a drop of rain
So let the rain come on
Outside, outside comes the sun

Hard by design
Hard times
Are just a drop of rain
So let the rain come on
Outside, outside runs the sun


## Get starred songs

In [1]:
# import require modules
import sys
import spotipy
import spotipy.util as util
import pandas as pd

# spotify api parameters
username = 'colinspear1'
client_id='ce2cc5f48d51431d80ff674bfba165e5'
client_secret='e36aec36c3174decbce246226e6d8b7a'
redirect_uri = 'http://localhost/'
scope = 'user-library-read'

# get token
token = util.prompt_for_user_token(
    username=username,
    scope=scope, 
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri
)

# instantiate spotipy
sp = spotipy.Spotify(auth=token)

In [2]:
# collect saved tracks
# improve by not limiting result to 350 - extend indefinitely
song_list = []
i = 0
while i < 350:
    results = sp.current_user_saved_tracks(limit=50, offset=i)
    for item in results['items']:
        song_list.append(item)
    i += 50

## Save song features and analysis

In [68]:
# initiate song dictionary
song_dict = dict(
    time_added=[],
    release_date=[],
    release_date_precision=[],
    artist_name=[],
    artist_id=[],
    song_id=[],
    song_length_ms=[],
    song_name=[],
    popularity=[], 
    loudness = [], 
    tempo = [], 
    tempo_confidence = [], 
    time_signature = [],
    time_sig_conf = [], 
    key = [], 
    key_confidence = [], 
    mode = [], 
    mode_confidence = [], 
    danceability = [],
    energy = [], 
    speechiness = [], 
    acousticness = [], 
    instrumentalness = [], 
    liveness = [], 
    valence = [], 
)

# collect saved track data
for i, song in enumerate(song_list):
    if song_list[i]['track']['id'] in song_dict['song_id']:
        continue
    else:        
        if i % 25 == 0:
            print('Grabbing data from song number {}'.format(i))
        
        song_dict['time_added'].append(song_list[i]['added_at'])
        song_dict['release_date'].append(song_list[i]['track']['album']['release_date'])
        song_dict['release_date_precision'].append(song_list[i]['track']['album']['release_date_precision'])
        song_dict['artist_name'].append(song_list[i]['track']['album']['artists'][0]['name'])
        song_dict['artist_id'].append(song_list[i]['track']['artists'][0]['id'])
        song_dict['song_id'].append(song_list[i]['track']['id'])
        song_dict['song_length_ms'].append(song_list[i]['track']['duration_ms'])
        song_dict['song_name'].append(song_list[i]['track']['name'])
        song_dict['popularity'].append(song_list[i]['track']['popularity'])
 
        audio_analysis = sp.audio_analysis(song_list[i]['track']['id'])
        song_dict['loudness'].append(audio_analysis['track']['loudness'])
        song_dict['tempo'].append(audio_analysis['track']['tempo'])
        song_dict['tempo_confidence'].append(audio_analysis['track']['tempo_confidence'])
        song_dict['time_signature'].append(audio_analysis['track']['time_signature'])
        song_dict['time_sig_conf'].append(audio_analysis['track']['time_signature_confidence'])
        song_dict['key'].append(audio_analysis['track']['key'])
        song_dict['key_confidence'].append(audio_analysis['track']['key_confidence'])
        song_dict['mode'].append(audio_analysis['track']['mode'])
        song_dict['mode_confidence'].append(audio_analysis['track']['mode_confidence'])
        
        audio_features = sp.audio_features(top_song_id)
        song_dict['danceability'].append(audio_features[0]['danceability'])
        song_dict['energy'].append(audio_features[0]['energy'])
        song_dict['speechiness'].append(audio_features[0]['speechiness'])
        song_dict['acousticness'].append(audio_features[0]['acousticness'])
        song_dict['instrumentalness'].append(audio_features[0]['instrumentalness'])
        song_dict['liveness'].append(audio_features[0]['liveness'])
        song_dict['valence'].append(audio_features[0]['valence'])
        
fave_songs = pd.DataFrame(song_dict)
fave_songs.to_pickle('starred_songs.pkl')

Grabbing data from song number 0
Grabbing data from song number 25
Grabbing data from song number 50
Grabbing data from song number 75
Grabbing data from song number 100
Grabbing data from song number 125
Grabbing data from song number 150
Grabbing data from song number 175
Grabbing data from song number 200
Grabbing data from song number 225
Grabbing data from song number 250
Grabbing data from song number 275
Grabbing data from song number 300


## Most popular of saved songs

In [67]:
fave_songs = pd.DataFrame(song_dict)

In [70]:
pop_max = fave_songs['popularity'].max()
pop_cond = fave_songs.index[fave_songs['popularity'] == pop_max]
top_song_artist = fave_songs.loc[pop_cond, 'artist_name'].values[0]
top_song_name = fave_songs.loc[pop_cond, 'song_name'].values[0]
top_song_id = fave_songs.loc[pop_cond, 'song_id'].values[0]

print('The most popular song in your catalog is {} by {} with a popularity score of {}.'.format(top_song_name, top_song_artist, pop_max))

The most popular song in your catalog is Herside Story by GoldLink with a popularity score of 70.


In [62]:
audio_analysis = sp.audio_analysis(top_song_id)
audio_features = sp.audio_features(top_song_id)

In [65]:
audio_features[0]['danceability']

0.295

In [44]:
section_dict = dict(
    start = []
)
for section in audio_analysis['sections']:
    section_dict['start'].append(section['start'])

In [45]:
section_dict

{'start': [0.0, 7.41373, 32.85152, 47.87509, 68.33941, 101.31868, 125.10809]}

In [ ]:
fave_songs = pd.DataFrame(song_dict)

In [19]:
audio_analysis['segments'][0]

{'start': 0.0,
 'duration': 0.66236,
 'confidence': 0.0,
 'loudness_start': -60.0,
 'loudness_max_time': 0.0,
 'loudness_max': -60.0,
 'pitches': [0.075,
  0.073,
  0.096,
  0.05,
  0.072,
  0.052,
  0.057,
  0.078,
  0.045,
  0.066,
  1.0,
  0.954],
 'timbre': [0.0,
  171.13,
  9.469,
  -28.48,
  57.491,
  -50.067,
  14.833,
  5.359,
  -27.228,
  0.973,
  -10.64,
  -7.228]}

In [17]:
for section in audio_analysis['sections']:
    print(section['start'])

0.0
13.04162
37.47592
161.85907
175.73222
201.31659
218.86235
309.7091


In [18]:
audio_features

[{'danceability': 0.683,
  'energy': 0.842,
  'key': 0,
  'loudness': -7.075,
  'mode': 1,
  'speechiness': 0.0626,
  'acousticness': 0.214,
  'instrumentalness': 0.539,
  'liveness': 0.192,
  'valence': 0.777,
  'tempo': 81.972,
  'type': 'audio_features',
  'id': '0PiKRl8dvzGrxAfbwKUR8w',
  'uri': 'spotify:track:0PiKRl8dvzGrxAfbwKUR8w',
  'track_href': 'https://api.spotify.com/v1/tracks/0PiKRl8dvzGrxAfbwKUR8w',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0PiKRl8dvzGrxAfbwKUR8w',
  'duration_ms': 316720,
  'time_signature': 4}]

In [26]:
dict(
    danceability = [],
    energy = [], 
    key = [], 
    loudness = [], 
    mode = [], 
    speechiness = [], 
    acousticness = [], 
    instrumentalness = [], 
    liveness = [], 
    valence = [], 
    tempo = [], 
    time_signature = []
)


1
22
333
4444
55555
666666
7777777
88888888
999999999
